In [1]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

In [6]:
pd.read_csv('./eyeShadow.csv').head()

,userId,name,popId,rate,review,type
0,0,매트 아이 컬러,0,4.57,진저 진짜 짱 추천합니다! 코섀디에도 완전 자연스럽고 그냥 단독으로도 진짜 이뻐요?...,건성
1,1,매트 아이 컬러,0,4.57,진저....평생이걸로쓸거에요,건성
2,2,매트 아이 컬러,0,4.57,완전 매트하고 다 제가 좋아하는 색이여서 너무 좋아요!,건성
3,3,매트 아이 컬러,0,4.57,좋은 거 하나 사서 힛팬 하는 걸 목표로 프레스코 구매함. 프레스코 저렴이 많이 가...,건성
4,4,매트 아이 컬러,0,4.57,한창 섀도우에 빠져 있을때 뭐 살지 고민하다가 평이 좋아서 진저 구매해서 사용해 봤...,건성
